# Similarity of documents

In the [data analysis and processing directory](./02_Data_Analysis_And_Processing) we created a vector to associate with each article which, in some sense, attempts to capture the content of the article (or at least the content of the abstract). So when we talk about whether or not two articles are similar, we're really talking about the similarity of the vectors. A standard way of evaluating the similarity between two vectors is looking at the angle between them. This should be motivated by the fact that parallel vectors seem like they should be similar while orthogonal vectors are dissimilar. Numerically we represent this quantity by considering the normalized dot product of the two vectors (this number, normalized correctly, is the cosine of the angle between the vectors). 

The normalized dot product of two vectors can take values between $-1$ and $1$, with $1$ corresponding to parallel vectors (high similarity of articles) and $-1$ corresponding to vectors pointing in opposite directions (very dissimilar articles). All of the scores we use refer to the cosine similarity. 

## Prep the data for computation

We use `numpy` to perform fairly efficient numerical matrix operations and some sorting. These mathematical operations translate, from the lens of recommending articles to a user, to computing similarity scores and then sorting the articles by their score. 

The `json` and `pickle` libraries are used to persist the objects used in these computations, since we will need to reuse them to update the database. Also while preprocessing is running, we can have the objects stored in memory on the Flask app serving an API which will allow us to compute recommendations online. 

The `csv` module is used to simplify reading the `CSV` file which stores the article vectors.

The `pandas` module is used here to examing some sample recommendations.

For the sake of storage and memory, we'll only keep the top 50 rated recommendations for each article.

In [1]:
import numpy as np
import json
import pickle
import csv


In [2]:
def split_vectors_ids(vectors_file='../../vectors/arxiv_vectors.csv',
                      id_json='../../vectors/id.json',
                      vectors_pkl='../../vectors/vectors.pkl',
                     ):
    """
    Takes in the total csv of vectors for 
    the articles, indexed by arxiv_id and 
    transforms them into a list of ids
    and array of vectors. Then the array is 
    pickled and the ids list is stored as 
    a json.
    """
    
    ids = []
    vectors = []
    
    with open(vectors_file, 'r', newline='') as vectors_csv:
        vectors_reader = csv.reader(vectors_csv)
        for id, *vector in vectors_reader:
            ids.append(id)
            vector = np.array([float(component) for component in vector])
            vectors.append(vector)
    
    with open(id_json, 'w') as json_file:
        json.dump(ids, json_file)
    
    vectors = np.array(vectors)
    vectors = normalize_rows(vectors)
    
    with open(vectors_pkl, 'bw') as pkl_file:
        pickle.dump(vectors, pkl_file)

In [3]:
# split_vectors_ids()

There is a bug with loading large pickled files on MacOS, so id you try to run this locally on a Mac, it probably won't work!

In [4]:
with open('../../vectors/vectors.pkl', 'rb') as vec_pkl:
    vectors = pickle.load(vec_pkl)
    
with open('../../vectors/id.json', 'r') as ids_json:
    all_ids = json.load(ids_json)

## Computing the Scores

The functions in this subsection serve the following purpose

1. `score` - compute a batch similarity scores 
1. `sort_scores` - sort the similarity scores, keeping in mind which articles the scores are associated with
1. `compute_recs_paper_id` - A convenience function for computing the recommendations of an article by specifying the id of the article. This will be used in the Flask app for online computation of recommendations.

In [5]:
def score(vectors, all_ids, id_low, id_high):
    """
    Outputs an array, scores. The columns of scores
    are correspond to the slice of article_ids

    all_id[id_low:id_high]

    The rows are indexed by all_ids.

    The value in column C and row R is the
    similarity between the articles all_ids[R]
    and all_ids[id_low+C].
    """
    mask = [id_low <= index < id_high for index, _ in enumerate(all_ids)]
    rows = vectors[mask]
    scores = vectors @ rows.T
    return scores

In [6]:
def sort_scores(scores, all_ids, cur_ids):
    """
    Returns a dictionary of lists. The keys are the ids
    of articles we're currently evaluating.

    The lists contain tuples of scores and ids. The
    score is the similarity score between the current article
    and the other component of the tuple.
    """
    recs_index = scores.argsort(0)[-51:,:][::-1]
    recs = {
      id:[(all_ids[index], scores[index][0]) for index in recs_index[:, col_num]]
        for col_num, id in enumerate(cur_ids)
    }
    return recs

In [7]:
def compute_recs_batch(all_ids, vectors, low, high):
    """
    Computes the recommendations for the articles in
    the list all_ids[low:high].
    Returns a dictionary that can be fed into
    send_to_server for upload to the postgres database.
    """
    scored = score(vectors, all_ids, low, high)
    recs = sort_scores(scored, all_ids, all_ids[low:high])
    return recs

In [10]:
%time recs = compute_recs_paper_id('1801.08262', all_ids, vectors)

CPU times: user 800 ms, sys: 0 ns, total: 800 ms
Wall time: 533 ms


In [11]:
len(recs['1801.08262'])

51

In [12]:
recs['1801.08262'][:5]

[('1801.08262', 1.0000000000000002),
 ('1212.2697', 0.96994439808605737),
 ('1410.0230', 0.9675059791477697),
 ('1611.03840', 0.96315506293288022),
 ('1705.00164', 0.96170088971694145)]

### Interacting with SQL

We'll define some functions that allow us to store the recommendations in a database so we don't have to compute them again when they're requested by the flask app. In particular the flask app should be able to perform two specific operations with the database.

1. We should be able to retrieve check if a record exists in the table, and if it does render something for the user.
2. If there is not matching record in the table, we can update the table and then service the user.

In [13]:
from sqlalchemy_arxiv import Session, articles_similar

In [16]:
def send_to_server(recs, table_class_recs, session):
    """
    Sends computed recommendations to the the database
    """
    new_recs = [{'id':key,'recs':value} for key, value in recs.items()]
    new_recs = [table_class_recs(**args) for args in new_recs]
    for new in new_recs:
        session.merge(new)
    session.commit()


Send the new recommendations to the server.

In [21]:
session = Session()
send_to_server(recs, articles_similar, session)
session.close()

Get the records back from the server.

In [23]:
def request_recs(id_request, table_class_recs, session):
    """
    Retrieves computed recommendations from the database
    """
    query = (session
             .query(table_class_recs)
             .filter(table_class_recs.id==id_request)
            )
    records = query.all()
    if records:
        return records


In [69]:
session = Session()
recs_ = request_recs('1801.08262', articles_similar, session)
session.close()

In [70]:
recs_[0].recs[:5]

[['1801.08262', 1.0000000000000002],
 ['1212.2697', 0.9699443980860574],
 ['1410.0230', 0.9675059791477697],
 ['1611.03840', 0.9631550629328802],
 ['1705.00164', 0.9617008897169415]]

## Random articles results

We'll take a concrete look at a random sample of recommendations of articles, and see how they stack up. First let's get a random articlea and look at their top 5 similar articles.

In [143]:
import pandas as pd

In [146]:
np.random.seed(40)
random_ids = np.random.choice(all_ids, size=1)

This is effectively a simulation of how we would respond to a request for a recommendation in practice. A user would provide an `arxiv_id` and we would respond with other `arxiv_ids`, hopefully as a link and with things like the title and abstract of the recommended articles.

In [150]:
random_ids[0]

'quant-ph/0307015'

In [151]:
random_recs = []
for id in random_ids:
    article_index = all_ids.index(id)
    tmp = score(vectors, all_ids, article_index,  article_index+1)
    tmp = sort_scores(tmp, all_ids, [id])
    tmp[id] = tmp[id][:6]
    random_recs.append(tmp)

In [152]:
rec_dict = random_recs[0]
pd.Series(data=[x[1] for x in list(rec_dict.values())[0]], index=[x[0] for x in list(rec_dict.values())[0]])

quant-ph/0307015    1.000000
quant-ph/0509075    0.908993
quant-ph/0406008    0.894806
quant-ph/0611137    0.891376
quant-ph/0207112    0.888474
quant-ph/9712020    0.887291
dtype: float64

If you follow any of the ids above to their pages on arxiv, ie go to https://arxiv.org/abs/quant-ph/0307015, you will find the following titles:
1. [Bounds on the Probability of Success of Postselected Non-linear Sign Shifts Implemented with Linear Optics](https://arxiv.org/abs/quant-ph/0307015)
1. [Feed-forward and its role in conditional linear optical quantum dynamics](https://arxiv.org/abs/quant-ph/0509075)
1. [An efficient quantum filter for multiphoton states](https://arxiv.org/abs/quant-ph/0406008)
1. [Minimum-energy pulses for quantum logic cannot be shared](https://arxiv.org/abs/quant-ph/0611137)
1. [Linear optics implementation of general two-photon projective measurement](https://arxiv.org/abs/quant-ph/0207112)
1. [Optimal Signal-to-Quantum Noise Ratio for Nonclassical Number States](https://arxiv.org/abs/quant-ph/9712020)

Keep in mind, the first article in this list is the article we're providing recommendations for. A quick read of the abstracts of these articles tells me that they're all related to optics and and photon gates. I would say these recommendations are good, and that these paper are certainly similar, but I can't really judge how perfectly aligned they are. 

## Testing a non-random article
There are some articles I can personally judge, as a subject matter expert, as being good recommendations. For example, there is a paper I wrote recently, with my former advisor Sergi Elizalde, titled [Wilf equivalence relations for consecutive patterns](https://arxiv.org/abs/1801.08262). Let's look at the recommendations this system provides for that paper.

In [157]:
my_paper = '1801.08262'
article_index = all_ids.index(my_paper)
tmp = score(vectors, all_ids, article_index,  article_index+1)
tmp = sort_scores(tmp, all_ids, [my_paper])
tmp[my_paper] = tmp[my_paper][:6]

In [159]:
rec_dict = tmp
pd.Series(data=[x[1] for x in list(rec_dict.values())[0]], index=[x[0] for x in list(rec_dict.values())[0]])

1801.08262    1.000000
1212.2697     0.969944
1410.0230     0.967506
1611.03840    0.963155
1705.00164    0.961701
0805.1872     0.961022
dtype: float64

We do the same thing we did above, the following list of articles begins with the article we're finding recommendations for, and the remainder of the list are the recommended articles. I can, without qualificiation, say that these are good recommendations. All of these papers absolutely are a part of a topic in combinatorial mathematics called __permutation patterns__. This is a very specialized sub-field of mathematics, and the recommender is definitely staying within this sub-field. 

1. [Wilf equivalence relations for consecutive patterns](https://arxiv.org/abs/1801.08262)
1. [On Pattern Avoiding Alternating Permutations](https://arxiv.org/abs/1212.2697)
1. [Egge triples and unbalanced Wilf-equivalence](https://arxiv.org/abs/1410.0230)
1. [The Length of the Longest Common Subsequence of Two Independent Mallows Permutations](https://arxiv.org/abs/1611.03840)
1. [Quadrant marked mesh patterns in 123-avoiding permutations](https://arxiv.org/abs/1705.00164)
1. [Avoidance of Partially Ordered Generalized Patterns of the form  k-σ-k](https://arxiv.org/abs/0805.1872)

## A non-example

There's no way the recommender will always give results as good as the ones above. Finding an example of this can be a little complicated. Below I compute recommendations for an article that I read in graduate school and genuinely had trouble making sense of and tried to find other sources. 

In [161]:
my_paper = 'math/9806036'
article_index = all_ids.index(my_paper)
tmp = score(vectors, all_ids, article_index,  article_index+1)
tmp = sort_scores(tmp, all_ids, [my_paper])
tmp[my_paper] = tmp[my_paper][:6]

In [162]:
rec_dict = tmp
pd.Series(data=[x[1] for x in list(rec_dict.values())[0]], index=[x[0] for x in list(rec_dict.values())[0]])

math/9806036    1.000000
1106.5646       0.847961
cs/0512073      0.820467
1106.5531       0.818694
1401.1089       0.814763
0801.3194       0.803516
dtype: float64

1. [The Goulden-Jackson Cluster Method: Extensions, Applications and Implementations](https://arxiv.org/abs/math/9806036)
1. [The Number of Same-Sex Marriages in a Perfectly Bisexual Population is Asymptotically Normal](https://arxiv.org/abs/1106.5646)
1. [Schwerdtfeger-Fillmore-Springer-Cnops Construction Implemented in GiNaC](https://arxiv.org/abs/cs/0512073)
1. [Balls in Boxes: Variations on a Theme of Warren Ewens and Herbert Wilf](https://arxiv.org/abs/1106.5531)
1. [Automatic Enumeration of Generalized Menage Numbers](https://arxiv.org/abs/1401.1089)
1. [The Fedosov *-product in Mathematica](https://arxiv.org/abs/0801.3194)

The scores here are much lower than the previous examples, which we would hope indicates that the matches are not as good. This is indeed the case! Looking at the abstracts of these papers nothing about the content that indicates a strong relationship between these articles and the the article we're finding recommendations for. There is however on striking similarity in all of these abstracts. The all contain the phrase `this http url` in reference to a link. I have no idea if this is some old school way of describing links, but this idiom really seems to have boosted the scores between these articles. 